In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python
!pip install datasets

In [ ]:
from datasets import load_dataset
SETTING = 'vision'
dataset = load_dataset('MMMU/MMMU_Pro', SETTING, split='test')
image = dataset[0]['image']
display(image)

In [14]:
import sys
sys.path.append(r"./python")

import os
import json
from model import *

#根据时间情况修改index和language值
index =  "wy_demo_image"
embedding_endpoint_name = "cohere.embed-multilingual-v3"

embedding_type = 'bedrock' if embedding_endpoint_name.find('titan') or embedding_endpoint_name.find('cohere') else 'sagemaker'
embeddings = init_embeddings_bedrock(embedding_endpoint_name)

In [ ]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
import fitz
from PIL import Image
import numpy as np
import base64
from opensearch_multimodel_dataload import add_multimodel_documents
import re
import os
import time

model_name = "anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
llm = init_model_bedrock(model_name)
text_max_length = 2000

prompt = """
brief the key search terms of the image. 
Keep the key search terms as simple as possible
No preface, just output the key search terms directly.
"""
llm_max_size = 800
display_max_size = 480

texts = []
metadatas = []

for i in tqdm(range(len(dataset))):
    if i < 21:
        continue
    print('sleep 60 sec')
    time.sleep(60)
    image = dataset[i]['image']
    image_id = dataset[i]['id']
    answer = dataset[i]['answer']
    subject = dataset[i]['subject']
    
    print(image.size)
    if image.size[0] > llm_max_size:
        image_for_llm = image.resize((llm_max_size,int(image.size[1]*llm_max_size / image.size[0])))
        print('resize image size:',image_for_llm.size)
    else:
        image_for_llm = image
    rgb_im = image_for_llm.convert('RGB')
    rgb_im.save('for_llm.jpg')
    
    if image.size[0] > display_max_size:
        image_for_display = image.resize((display_max_size,int(image.size[1]*display_max_size / image.size[0])))
        print('store image size:',image_for_display.size)
    else:
        image_for_display = image
    rgb_im = image_for_display.convert('RGB')
    rgb_im.save('for_display.jpg')
    
    with open("for_llm.jpg", "rb") as image_file:
        image_bytes = image_file.read()
        encoded_image = base64.b64encode(image_bytes).decode("utf-8")
        print('for_llm len:',len(encoded_image))
        model_kwargs = {'image': encoded_image,'max_tokens':2048,'language':'english'}
        llm.model_kwargs = model_kwargs
        response = llm(prompt=prompt)
        response = response.strip()
        texts.append(response)
        print('response:',response)
    
    
    with open("for_display.jpg", "rb") as image_file:
        image_bytes = image_file.read()
        encoded_image = base64.b64encode(image_bytes).decode("utf-8")
        print('for_display len:',len(encoded_image))
        base_64_head = 'data:image/jpg;base64,'
        image_code = base_64_head + encoded_image
        
    metadata = {}
    metadata['sentence'] = response[:text_max_length] if len(response) > text_max_length else response
    metadata['id'] = image_id
    metadata['answer'] = answer
    metadata['subject'] = subject
    metadata['image_code'] = image_code
    metadatas.append(metadata)
    os.remove('for_llm.jpg')
    os.remove('for_display.jpg')
    
    if len(texts) > 0:
        if embedding_type == 'bedrock':
            text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
        else:
            text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas],chunk_size=10)

        print('texts len:',len(texts))
        print('metadatas len:',len(metadatas))
        print('embeddings len:',len(text_embeddings))
        print('begin to save in vectore store')
        add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas
        )
        print('finish save in vectore store:',index)
        texts = []
        metadatas = []


  0%|          | 0/1730 [00:00<?, ?it/s]

sleep 60 sec
(562, 1181)
store image size: (480, 1008)
for_llm len: 237992
max_tokens: 2048
modelId: anthropic.claude-3-5-sonnet-20240620-v1:0
temperature: 0.01
language: english
response: neural pathways, senses, brain, visual processing, rods and cones, cerebral cortex, optic nerve, thalamus, dorsal and ventral pathways, neglect syndrome, visual agnosia, discrimination test, tracing test, auditory pathway
for_display len: 176272
texts len: 1
metadatas len: 1
embeddings len: 1
begin to save in vectore store


  1%|▏         | 22/1730 [01:03<1:22:41,  2.90s/it]

finish save in vectore store: wy_demo_image_2
sleep 60 sec
(1119, 1055)
resize image size: (800, 754)
store image size: (480, 452)
for_llm len: 48952
max_tokens: 2048
modelId: anthropic.claude-3-5-sonnet-20240620-v1:0
temperature: 0.01
language: english
response: abdominal x-ray, radiograph, abnormal organ, medical imaging, multiple choice question, anatomy, digestive system, skeletal structure
for_display len: 21928
texts len: 1
metadatas len: 1
embeddings len: 1
begin to save in vectore store


  1%|▏         | 23/1730 [02:06<3:05:20,  6.51s/it]

...

finish save in vectore store: wy_demo_image_2
sleep 60 sec
(2252, 2236)
resize image size: (800, 794)
store image size: (480, 476)
for_llm len: 120948
max_tokens: 2048
modelId: anthropic.claude-3-5-sonnet-20240620-v1:0
temperature: 0.01
language: english
response: rat experiment, green button, food pellet, wait time, reinforcement, behavioral psychology, graph, trial number, button presses
for_display len: 56452
texts len: 1
metadatas len: 1
embeddings len: 1
begin to save in vectore store


 13%|█▎        | 225/1730 [3:31:35<25:57:30, 62.09s/it]

finish save in vectore store: wy_demo_image_2
sleep 60 sec
(1280, 960)
resize image size: (800, 600)
store image size: (480, 360)
for_llm len: 64232
max_tokens: 2048
modelId: anthropic.claude-3-5-sonnet-20240620-v1:0
temperature: 0.01
language: english
response: literary device, text and illustration, different stories, Rosie's Walk, cat illustration, multiple choice question, juxtaposition, metaphor, paradox, allegory, contrast, reasoning, satire, printmaking, irony, counterpoint
for_display len: 25928
texts len: 1
metadatas len: 1
embeddings len: 1
begin to save in vectore store


 13%|█▎        | 226/1730 [3:32:37<26:01:44, 62.30s/it]

finish save in vectore store: wy_demo_image_2
sleep 60 sec
